In [1]:
import pandas as pd


# Sources

NBC News is publishing its database of more than 200,000 tweets that Twitter has tied to "malicious activity" from Russia-linked accounts during the 2016 U.S. presidential election.


https://developers.slashdot.org/story/18/02/17/0038251/nbc-publishes-200000-tweets-tied-to-russian-trolls



The original file has some junk in it which we have to delete before opening with Pandas :

Tweets from confirmed Russian trolls, shows only username, timestamp (in UTC), tweet text, and number of times tweet was retweeted and favorited according to our data",,,,,,,,,,,,,,,,,

From NBC News' story: https://www.nbcnews.com/tech/social-media/now-available-more-200-000-deleted-russian-troll-tweets-n844731,,,,,,,,,,,,,,,,,

"If you publish using the data, please credit NBC News and include a link to this page. Send questions to ben.popken@nbcuni.com or twitter.com/bpopken.",,,,,,,,,,,,,,,,,

Twitter screenname,Date tweet sent,Tweet text,Times retweeted,Times favorited,,,,,,,,,,,,,

ryanmaxwell_1,3/22/2016 18:31,#IslamKills Are you trying to say that there were no terrorist attacks in Europe before refugees were let in?,,,,,,,,,,,,,,,



In [ ]:
fp = file('streamlined_tweets.csv','r')
print ''.join(fp.readlines()[:5])

In [86]:
fp = file('streamlined_tweets.csv','r')
fe = file('fakenews_tweets.csv','w')

with fe as piggy:
    for L in fp.readlines()[3:]:
        piggy.write(L)

df = pd.read_csv('fakenews_tweets.csv')

so the data now looks OK 
there are still a few NaNs in the **Tweet text**
column but this isn't so important

In [91]:
' '.join( list( df.columns.values)[:5])

'Twitter screenname Date tweet sent Tweet text Times retweeted Times favorited'

In [88]:
df[:5]

,Twitter screenname,Date tweet sent,Tweet text,Times retweeted,Times favorited,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,ryanmaxwell_1,3/22/2016 18:31,#IslamKills Are you trying to say that there w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,detroitdailynew,10/10/2016 20:57,"Clinton: Trump should’ve apologized more, atta...",0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cookncooks,2/22/2017 12:43,RT @ltapoll: Who was/is the best president of ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,queenofthewo,12/26/2016 15:06,RT @jww372: I don't have to guess your religio...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mrclydepratt,8/6/2017 2:36,RT @Shareblue: Pence and his lawyers decided w...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Rough Analysis

This is just screwing around to see what is in the file.

### Todo

I should really feed this through an **NLP** pipeline.

In [60]:
AA = df[['Twitter screenname']].values
BB = df[['Tweet text']].values

In [58]:
names = set( AA.reshape((-1,)))

In [63]:
txts =  list(BB.reshape((-1,)))

In [72]:
txts = [x.replace('\n',' ') + '\n' for x in txts if type(x) == str]

so here there are a load of texts about Pokemon.

- What does this prove? 
- Why didn't they clean the data?
- Do those pesky Russians spies want to prove they are real people?

In [92]:
pokemon = [x for x in txts if 'pokemon' in x.lower()]
len(pokemon)

181

In [93]:
`print ''.join(pp)

#2017SurvivalTips never ever talk about Pokemon Gp
#IHatePokemonGoBecause it's the precursor to the zombie apocalypse.
RT @davethedave_: #IProcrastinateBy creating spreadsheets when I should be catching Pokemon.
RT @CoffeeNTrees: #IHatePokemonGoBecause It gives meth heads a reason to leave their house at 3am.
#IHatePokemonGoBecause it makes me look like a creep. https://t.co/vhuXPSmciG
RT @Ely_K_Werdel: #IHatePokemonGoBecause nobody ever searches my yard when the bears are around
RT @SPIEGELONLINE: In Dänemark hat eine 49-Jährige bei der Pokémon-Jagd eine Leiche entdeckt. #PokemonGo https://t.co/e3zaapbTXb
RT @lonly_pram: #IHatePokemonGoBecause Tired of their constant ?s: Why is the sky blue? Is time even real? Why doesn't my mom love me? Mayb…
#IHatePokemonGoBecause it feels like I live in Japan https://t.co/5pDWImI1YI
 Pokemon/illegal dog fighting rings
RT @tracyreson: #IHatePokemonGoBecause its not in my country,sucks
RT @googergieger: #IHatePokemonGoBecause it doesn't love me back!

# Hashtag analysis

I'm not going to do anything complicated but here we go

In [144]:
import re
#OK so it's simple and not magic but it works
ph = re.compile(r'\#(\w*)')

In [111]:
data = file('amalgam_txts.txt','r').read()

In [145]:
tags = ph.findall(data)
tags = list(set(tags))
tags.sort()

In [247]:
clinton = [x for x in tags if 'clinton' in x.lower()]
clinton = list(set(clinton))
clinton.sort()

In [248]:
associated_words(clinton,['clintons','hilary'])

{'007movies',
 '3strikes',
 'abovethelaw',
 'afilm',
 'anybodybut',
 'arrest',
 'assad',
 'body',
 'bodycount',
 'bothsare',
 'cages',
 'campaign',
 'canttrusts',
 'cash',
 'chelsea',
 'chelseaemails1',
 'chinatech',
 'collap',
 'collapse',
 'contamination',
 'correctional',
 'corrupt',
 'corruption',
 'corrupts',
 'cri',
 'crime',
 'crimecartel',
 'crimefamily',
 'crimefound',
 'crimefoundation',
 'crimes',
 'crimesyn',
 'crimesyndicate',
 'crimewave',
 'criminal',
 'criminalcartel',
 'crooked',
 'crookeds',
 'cult',
 'debatedemands',
 'dropouttrumpparty',
 'email',
 'emails',
 'emailscandal',
 'essential',
 'fbinotes',
 'fdn',
 'fluffer',
 'fluffers',
 'forpresident',
 'foundation',
 'foundationcorruption',
 'girlsbreaktheinternet',
 'hate',
 'health',
 'hillory',
 'i',
 'imelda',
 'impeach',
 'indict',
 'isar',
 'isarapist',
 'isarapistenabler',
 'israpist',
 'isthenewcword',
 'kaine',
 'kid',
 'kill',
 'killary',
 'kills',
 'koolade',
 'lies',
 'liesmatter',
 'machine',
 'mail',
 '

In [158]:
bill = [x for x in clinton if 'bill' in x.lower()]
hil = [x for x in clinton if 'hillary' in x.lower() ]

we have to write a filter to clean things up 
using a list of **stop words**

In [234]:
def associated_words(tags, omit=['']):
    stop_words = omit + ['bill','hillary','clinton','2016','16','_'] 
    old = ' '.join(tags)
    stuff = old.lower()
    for x in stop_words:
        stuff = stuff.replace(x,'')
    return set(stuff.split())

In [233]:
associated_words(hil)

{'arrest',
 'corrupt',
 'criminal',
 'crooked',
 'forpresident',
 'indict',
 'rotten',
 'scripted',
 'searchterms',
 'theliesof'}

In [228]:
obama = [x for x in tags if 'obama' in x.lower()]

In [235]:
associated_words(obama, omit=['obama','barack'])

{'44',
 '4prison',
 '7wars',
 '99daysof',
 'admin',
 'administration',
 'arrest',
 'arrestnow',
 'b',
 'bibivs',
 'birdlaunchby',
 'birthday',
 'birther',
 'bomb',
 'bye',
 'byebye',
 'ca',
 'care',
 'carefail',
 'careinthreewords',
 'coalition',
 'corrupt',
 'countdown',
 'criesabout',
 'crooked',
 'dance',
 'day',
 'defeat',
 'fail',
 'failed',
 'failedhopeandchange',
 'failedus',
 'family',
 'farew',
 'farewell',
 'farewelladdress',
 'farewells',
 'finaldays',
 'foreignpolicy',
 'forprison',
 'fuck',
 'g',
 'gate',
 'goodbye',
 'goodjob',
 'goodriddance',
 'growapair',
 'happy',
 'happybirthday',
 'happynday',
 'hometown',
 'husseininsane',
 'ilove',
 'imgoingtomissbecause',
 'impeach',
 'impeachnow',
 'imprison',
 'indict',
 'insult',
 'isafraud',
 'iscancer',
 'isisstra',
 'isisstrategies',
 'istheproblem',
 'istheworstpresidentever',
 'jail',
 'king',
 'knew',
 'legacy',
 'legacyoffailures',
 'lies',
 'malia',
 'michelle',
 'michellemade',
 'n',
 'nation',
 'ncare',
 'nextjob',
 

In [241]:
america = [x for x in tags if 'america' in x.lower()]
associated_words(america, omit=['american','america'])

{'1st',
 '4',
 '4alls',
 'actfor',
 'african',
 'africanhomelandstates',
 'africans',
 'allrejects',
 'anti',
 'antirhetoric',
 'antitrumpis',
 'bendover',
 'black',
 'blackpbs',
 'blacks',
 'bornagain',
 'callbank',
 'cgi',
 'contractwith',
 'dad',
 'death2',
 'deathto',
 'decides',
 'democratsdestructionof',
 'deservesbetter',
 'disgraceto',
 'divided',
 'eagle',
 'eagleday',
 'energyplan',
 'f',
 'failedslogans',
 'fake',
 'falls',
 'feminine4trump',
 'fi',
 'fir',
 'firs',
 'first',
 'flag',
 'for',
 'fuckingsinrealtime',
 'fundamentallychanged',
 'getitright',
 'godbless',
 'gods',
 'gophatess',
 'gopunfit2leadorgoverns',
 'greats',
 'grindingdown',
 'guns',
 'gunsin',
 'hates',
 'hatess',
 'heartassociation',
 'hire',
 'historyisblack',
 'honest',
 'horrorstory',
 'hostage',
 'howtosurvivetrumps',
 'idol',
 'ilove',
 'include',
 'inspiring',
 'isback',
 'isdoomed',
 'ismnotglobalism',
 'keepbeautiful',
 'keepsafeagain',
 'lagalaxyvs',
 'lefthates',
 'letsmakegreat',
 'letsmakegre